In [21]:
!pip install dash
!pip install dash-bootstrap-components

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mat
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc


In [23]:
# Including Dataset from Drive
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/BIT/CO2_emission_by_countries.csv')
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Country,Code,Calling Code,Year,CO2 emission (Tons),Population(2022),Area,% of World,Density(km2)
0,Afghanistan,AF,93,1750,0.0,41128771.0,652230.0,0.40%,63/km²
1,Afghanistan,AF,93,1751,0.0,41128771.0,652230.0,0.40%,63/km²
2,Afghanistan,AF,93,1752,0.0,41128771.0,652230.0,0.40%,63/km²
3,Afghanistan,AF,93,1753,0.0,41128771.0,652230.0,0.40%,63/km²
4,Afghanistan,AF,93,1754,0.0,41128771.0,652230.0,0.40%,63/km²
...,...,...,...,...,...,...,...,...,...
59615,Zimbabwe,ZW,263,2016,736467042.0,16320537.0,390757.0,0.30%,42/km²
59616,Zimbabwe,ZW,263,2017,746048675.0,16320537.0,390757.0,0.30%,42/km²
59617,Zimbabwe,ZW,263,2018,757903042.0,16320537.0,390757.0,0.30%,42/km²
59618,Zimbabwe,ZW,263,2019,768852126.0,16320537.0,390757.0,0.30%,42/km²


In [24]:
#Delete rows with missing data
df = df.dropna(subset=["CO2 emission (Tons)","Year"])
#Making sure the variable Year is whole number
df ["Year"] = df ["Year"].astype(int)
df ["CO2 emission (Tons)"] = df ["CO2 emission (Tons)"].astype(float)

In [25]:
Countries = df["Country"].unique()
Years = df["Year"].unique()
emision_min = df["CO2 emission (Tons)"].min()
emision_max = df["CO2 emission (Tons)"].max()

In [26]:
display(df) #Information of Dataset

,Country,Code,Calling Code,Year,CO2 emission (Tons),Population(2022),Area,% of World,Density(km2)
0,Afghanistan,AF,93,1750,0.0,41128771.0,652230.0,0.40%,63/km²
1,Afghanistan,AF,93,1751,0.0,41128771.0,652230.0,0.40%,63/km²
2,Afghanistan,AF,93,1752,0.0,41128771.0,652230.0,0.40%,63/km²
3,Afghanistan,AF,93,1753,0.0,41128771.0,652230.0,0.40%,63/km²
4,Afghanistan,AF,93,1754,0.0,41128771.0,652230.0,0.40%,63/km²
...,...,...,...,...,...,...,...,...,...
59615,Zimbabwe,ZW,263,2016,736467042.0,16320537.0,390757.0,0.30%,42/km²
59616,Zimbabwe,ZW,263,2017,746048675.0,16320537.0,390757.0,0.30%,42/km²
59617,Zimbabwe,ZW,263,2018,757903042.0,16320537.0,390757.0,0.30%,42/km²
59618,Zimbabwe,ZW,263,2019,768852126.0,16320537.0,390757.0,0.30%,42/km²


In [27]:
display(df.describe()) #Showing the summary about the statistical datas
display(df.columns)

,Year,CO2 emission (Tons),Population(2022),Area
count,59620.000000,5.962000e+04,5.311600e+04,5.528400e+04
mean,1885.000000,1.031567e+09,3.992260e+07,6.522073e+05
std,78.231085,1.040504e+10,1.482365e+08,1.865483e+06
min,1750.000000,0.000000e+00,1.131200e+04,2.100000e+01
25%,1817.000000,0.000000e+00,1.770414e+06,1.770450e+04
50%,1885.000000,0.000000e+00,8.673095e+06,1.103815e+05
75%,1953.000000,8.715092e+06,2.862920e+07,4.925730e+05
max,2020.000000,4.170000e+11,1.425887e+09,1.709824e+07


Index(['Country', 'Code', 'Calling Code', 'Year', 'CO2 emission (Tons)',
       'Population(2022)', 'Area', '% of World', 'Density(km2)'],
      dtype='object')

In [28]:
# Creating app Dash with topic QUARTZ
external_stylesheets = [dbc.themes.QUARTZ]
app = Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "Emisiones globales de CO₂"

In [29]:
#Define the application layout
app.layout = dbc.Container([
    #Create a more beautiful and organized structure
    dbc.Row([html.H1("Global CO2 emissions")]),
    #Create a row with the title Global CO2 Emissions
    dbc.Row([
        dbc.Col([
            html.Label("Select a Country:"),
            #Text, filter
            dcc.Dropdown(
                id ='selector_country',
                #interactive selector, to show a list the options
                options = [{'label': Country, 'value': Country} for Country in Countries],
                #Generate an options list for to menú with info about the names countries
                value = df['Country'].unique()[0]
                #Unify the name of each country
        )]),

        dbc.Col([ #Creating colmn
            html.Label("Select the year range"), #Text
            dcc.RangeSlider(
            #Select a range the number like a year
                id = 'selector_year',
                #Component identifier.
                min = 1990, #Minimum value, far left
                max = 2020, #Maximum value, far right
                value = [1990, 2020],
                #Select the range between 1990 and 2020
                marks = {a: str(a) for a in range (1990, 2021)},
                #Create the labels by year
                step = 1
                #Advance year by year
        )]),

        dbc.Col([ #Creating colmn
            html.Label("Select emissions range (CO2):"), #Descriptive text
            dcc.RangeSlider(
                #Create a double slider, that is, one with two buttons that
                #allow you to select a range of values.
                id = 'selector_emision',
                #Component identifier
                min = emision_min, #Minimum allowed value
                max = emision_max, #Maximum allowed value
                value = [emision_min, emision_max], #Selected range
                marks =None, #Don't display any labels
                step = 1 # Increase by one unit

            )
         ])

    ]),
        dbc.Col([
            dbc.Col ([
                dcc.Graph(id = 'Graph_Bars_Emision_CO2')
            ]),
            dbc.Col([
                dcc.Graph(id = 'Graph_Cake_emision_CO2')
            ]),
            dbc.Col([
                dcc.Graph(id = 'Graph_Line_emision_CO2')
            ])
        ])

])

In [30]:
#Defining the callbacks
@app.callback(
    [Output('Graph_Bars_Emision_CO2', 'figure'), #Graphics to use with update
     Output('Graph_Cake_emision_CO2', 'figure'),
     Output('Graph_Line_emision_CO2', 'figure')], # Added comma here
    [Input('selector_country', 'value'), #Values ​​that the user selects
     Input('selector_year', 'value'),
     Input('selector_emision', 'value'),]
    # Removed Input('selector_feedback', 'value') as it's not defined in the layout
)
def update_graph(values_country, range_year, range_emision):
  #Filter the dataframe by the selected country
  df_filtered = df[df["Country"] == values_country ] # Corrected to use values_country
  df_filtered = df_filtered[(df_filtered["Year"] >= range_year[0]) & (df_filtered["Year"] <= range_year[1])] # Corrected year range
  #Filter only columns are in min and max values for the Year (1990-2020)
  df_filtered = df_filtered[(df_filtered["CO2 emission (Tons)"] >= range_emision[0]) & (df_filtered["CO2 emission (Tons)"] <= range_emision[1])]
  #Filter only CO2 emissions between min and max values

  #Using Plotly code here
  # ---------------------- GRAPH 1: BARS GRAPH ----------------------
  bars_graph = px.bar(
      df_filtered,
      x = "Year",
      y = "CO2 emission (Tons)",
      title= f"CO2 Emissions in {values_country}"
  )

  # ---------------------- GRAPH 2: CAKE GRAPH ----------------------
  cake_graph = px.pie(
      df_filtered,
      names = "Year",
      values = "CO2 emission (Tons)",
      title= f"Percentage of CO2 Emissions per year in {values_country}"
  )

  # ---------------------- GRAPH 2: LINE GRAPH ----------------------
  line_graph = px.line(
      df_filtered,
      x = "Year",
      y = "CO2 emission (Tons)",
      title= f"Evolution of CO2 Emissions in {values_country}"
  )

  for fig in [bars_graph, cake_graph, line_graph]:
      fig.update_layout(
          plot_bgcolor='rgba(0, 0, 0, 0)',
          paper_bgcolor='rgba(0, 0, 0, 0)'
      )
  return bars_graph, cake_graph, line_graph